In [ ]:
from snowflake.snowpark.context import get_active_session

# Set your database, schema, and virtual warehouse
DB = "DEMO"
SCHEMA = "PUBLIC"
COMPUTE_WAREHOUSE = "DEMO_WH"

session = get_active_session()
session.use_database(DB)
session.use_schema(SCHEMA)
session.use_warehouse(COMPUTE_WAREHOUSE)

In [ ]:
# Create a stage to store the images
CREATE OR REPLACE STAGE EXAMPLE_ITEMS 
	DIRECTORY = ( ENABLE = TRUE 
                  AUTO_REFRESH = TRUE) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

In [ ]:

# Upload the files to the stage using Snowpark
MY_STAGE = 'EXAMPLE_ITEMS/images'
MY_FILE_NAME = "images/*"

put_result = session.file.put(MY_FILE_NAME, MY_STAGE, auto_compress=False,overwrite=True)


In [ ]:
# Refresh the stage to reflect changes
ALTER STAGE EXAMPLE_ITEMS REFRESH;

In [ ]:
SELECT * FROM DIRECTORY(@EXAMPLE_ITEMS);

# Product Image Matching
The code below finds images that show the same or similar products

In [ ]:
-- Image matching with AI_FILTER. This is in Private Preview, so will require enablement from account team
WITH pictures AS (
  SELECT
      TO_FILE(file_url) AS img
  FROM DIRECTORY(@EXAMPLE_ITEMS)
)
SELECT self1.img, self2.img 
FROM pictures self1
JOIN pictures self2
ON AI_FILTER(PROMPT('Does the product shown in {0} match the product in {1}?', self1.img, self2.img));

In [ ]:
-- This approach uses AI_SIMILARITY, generating vector embeddings for each one
WITH pictures AS (
  SELECT
    relative_path,
      TO_FILE(file_url) AS img
  FROM DIRECTORY(@EXAMPLE_ITEMS)
)
SELECT
self1.img AS image_1,
self1.relative_path as image_1_path,
self2.img AS image_2,
self2.relative_path as image_2_path,
-- Semantic similarity score. Using 'voyage-multimodal-3' model
AI_SIMILARITY(self1.img, self2.img) AS sim_score,
FROM  pictures self1
CROSS JOIN pictures self2
WHERE sim_score < 1
QUALIFY ROW_NUMBER() OVER (PARTITION BY self1.relative_path
                         ORDER BY sim_score DESC) <= 3; -- Top 3

# Luxury good extraction
1. Start with filtering out images that have a luxury good
2. Then get details about the item

In [ ]:
-- Create dataset of images
WITH pictures AS (
  SELECT
      TO_FILE(file_url) AS img,
      relative_path
  FROM DIRECTORY(@EXAMPLE_ITEMS)
),
-- Filter to only real estate pictures with luxury goods
luxury_goods_only AS (
SELECT *
FROM pictures
WHERE AI_FILTER(PROMPT('Is this image {0} a real estate listing image with a luxury good?', pictures.img)
))
-- Extract information about that luxury good
SELECT * FROM luxury_goods_only;

In [ ]:
-- Create dataset of images
WITH pictures AS (
  SELECT
      TO_FILE(file_url) AS img,
      relative_path
  FROM DIRECTORY(@EXAMPLE_ITEMS)
),
-- Filter to only real estate pictures with luxury goods
luxury_goods_only AS (
  SELECT *
  FROM pictures
  WHERE AI_FILTER(PROMPT('Is this image {0} a real estate listing image with a luxury good?', pictures.img))
),
-- Extract information about that luxury good
luxury_items_classified AS (
  SELECT 
      *,
      -- Classify the luxury item into categories
      AI_CLASSIFY(
          img,
          ['painting', 'jewelry', 'musical instrument', 'sculpture', 'antique furniture', 'watch', 'collectible', 'other']
      ) AS item_class,

      AI_EXTRACT(
        file => img,
        responseFormat => [
            ['description', CONCAT('Give me a brief description of the luxury good pictured in this room. It belongs to the following category: ', item_class:labels[0]::STRING)]
        ]
      ) AS item_details
      
  FROM luxury_goods_only
)

SELECT 
    img,
    relative_path,
    item_class:labels[0]::STRING AS item_class,
    item_details:response
FROM luxury_items_classified;